In [7]:
import glob
import ete3
import pandas as pd
import os
import time
import json
from src import AFDB_tools, treescore , foldseek2tree
import colour
import numpy as np
import toytree
from matplotlib import pyplot as plt
from Bio import SeqIO
import toytree
import toyplot.svg

ModuleNotFoundError: No module named 'wget'

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
examples = './examples/*/'
download = False
for folder in glob.glob(examples):
    idfile = glob.glob(folder + '*identifiers.txt')[0]
    with open(idfile) as idin:
        identifiers = [l for l in idin]
    try:
        os.mkdir(folder + 'structs')
    except:
        for file in glob.glob(folder + 'structs/*.pdb'):
            os.remove(file)
    print('done setup')
    
    if download == True:
        notfound = [ AFDB_tools.grab_struct(uniID, folder + 'structs/')  for uniID in identifiers ]
        notfound = [ i for i in notfound if i]
        with open(folder+'unmapped.txt', 'w') as simout:
            simout.write(json.dumps(notfound))

In [ ]:
download = True
for t in glob.glob('./examples/sw'):
    print(t)
    try:
        os.mkdir(t+'structs/')
    except:
        print('already exists')
    outfolder = t+'structs/'
    
    if download == True:
        if 'pruned' not in t :
            print(t)
            tre = toytree.tree(t )
            notfound = [ AFDB_tools.grab_struct(uniID, outfolder)  for uniID in tre.get_tip_labels() ]
            notfound = [ i for i in notfound if i]
            with open(t+'unmapped.txt', 'w') as simout:
                simout.write(json.dumps(notfound))

In [ ]:
#prune not found
import ete3
for t in glob.glob('./ST_Update/*.nhx'):
    if 'pruned' not in t:
        with open(t+'unmapped.txt') as missing:
            missingIDs = json.loads(missing.read())
        print(missingIDs)
        print(t)
        tree = ete3.Tree(t)
        print(len(tree))
        keep = [ n for n in tree.traverse() if n.name not in missingIDs]
        tree.prune(keep, preserve_branch_length=True)
        print(len(tree))
        print(t+'pruned.nhx')
        with open(t+'pruned.nhx' , 'w' )as treeout:
            treeout.write(tree.write(format=1))

In [ ]:

for folder in glob.glob('./ST_Update/*/'):
    print(folder)
    input_structs = folder
    dbpath = foldseek2tree.runFoldseekdb(input_structs, folder)
    res = foldseek2tree.runFoldseek_allvall(dbpath , folder)

In [ ]:
for folder in glob.glob('./examples/*/'):
    print(folder)
    input_structs = folder+'structs/'
    dbpath = foldseek2tree.runFoldseekdb(input_structs, folder)
    res = foldseek2tree.runFoldseek_allvall(dbpath , folder)

In [2]:
def standard_treedraw( tre, sizes= None , colors= None ,fixed_order=None, fixed_position=None , ts = None,  save_file = False  , tiplabels = None):
    if tiplabels is None:
        tiplabels = tre.get_tip_labels()
    canvas, axes, mark = tre.draw(  
        ts = ts,
        node_sizes=sizes,
        node_colors=colors,
        tip_labels_align=True,
        scalebar=True,
        fixed_order=fixed_order, 
        fixed_position=fixed_position,
        tip_labels=tiplabels,
        tip_labels_style={
            "fill": "#262626",
            "font-size": "9px"}
    )
    if save_file:
        toyplot.svg.render(canvas, save_file)

In [3]:
#get trees
swisstreepath = './ST_Update/*nhxpruned.nhx'
swiss_trees = glob.glob(swisstreepath)
print(swiss_trees)
swiss_toytrees = {}

discretized_levels = 10
maxnodesize = 10
for t in swiss_trees:
    print(t)
    treecheck = True
    try:
        tre = toytree.tree(t , format =0 )
    except:
        print('tree err' , t)
        treecheck = False
    if treecheck == True:
        swiss_toytrees[t] = { 'tree' : tre }
        print(tre)
        swiss_toytrees[t]['labels'] = tre.get_tip_labels()
        swiss_toytrees[t]['coords'] = tre.get_tip_coordinates()
        swiss_toytrees[t]['path'] = t
        labels = swiss_toytrees[t]['labels']
        unidf = AFDB_tools.grab_entries(labels)
        lineages = treescore.make_lineages(unidf)
        discretized_levels = 10
        maxnodesize = 15
        taxlabels = dict(zip(unidf['query'] ,unidf.Organism) )
        #species_mapper = 
        tipnames = tre.get_tip_labels()
        taxnames = [ i+ ' '+taxlabels[i] if i in taxlabels else i for i in tipnames]
        red = colour.Color('blue')
        blue = colour.Color('red')
        tre = treescore.label_leaves(tre,lineages)
        color_vals = list(red.range_to(blue, discretized_levels+1))
        overlap = treescore.getTaxOverlap(tre.treenode)
        treevals = [ node.score for node in tre.treenode.traverse()]
        print(treevals)
        print('finalscore' , tre.treenode.score)
        swiss_toytrees[t]['score'] = tre.treenode.score
        treevals = tre.get_node_values('size', True, True)
        maxval = np.amax(treevals)
        minval =0
        bins = [minval + i*(maxval-minval)/discretized_levels for i in range(discretized_levels+1)]
        inds = np.digitize(treevals, bins)
        colors = [ color_vals[i-1].hex_l for i in list(inds)]
        sizes = [ (i / discretized_levels) * maxnodesize for i in list(inds) ]
        standard_treedraw(tre , sizes= sizes , colors=colors , tiplabels = taxnames , save_file = t + '.svg')
        standard_treedraw(tre  , sizes= sizes , colors=colors , tiplabels = taxnames )        

[]


In [4]:
import subprocess, shlex
def madroot(t):
    args = '/work/FAC/FBM/DBC/cdessim2/default/dmoi/software/mad/mad ' + t
    subprocess.run(shlex.split(args))
    return t+ '.rooted'

In [5]:

for folder in glob.glob('./examples/*/') :
    print(folder)
    treenumber = folder.split('/')[1]
    tmres = folder +'aln_score.tsv'
    res = pd.read_table(tmres, header = None ,delim_whitespace=True)
    res[0] = res[0].map(lambda x :x.replace('.pdb', ''))
    res[1] = res[1].map(lambda x :x.replace('.pdb', ''))
    self_dists = res[res[0] == res[1]]
    self_distmap = dict(zip(self_dists[0] , self_dists[2] ) )
    ids = list( set(list(res[0].unique()) + list(res[1].unique())))
    pos = { protid : i for i,protid in enumerate(ids)}
    print(res)
    distmat = np.zeros((len(pos), len(pos)))
    kernel_distmat = np.zeros((len(pos), len(pos)))
    for idx,row in res.iterrows():
        distmat[pos[row[0]] , pos[row[1]]]= row[3]
        kernel_distmat[pos[row[0]] , pos[row[1]]] = foldseek2tree.kernelfun(self_distmap[row[0]] , self_distmap[row[1]] , row[2])
    distmat = 1- distmat
    plt.imshow(distmat)
    plt.show()
    
    distmat_txt = foldseek2tree.distmat_to_txt( ids , distmat, folder , prefix ='standard' )
    distmat_txt_kernel = foldseek2tree.distmat_to_txt( ids , kernel_distmat , folder , prefix ='kernel' )
    out_tree = foldseek2tree.runFastme( 'fastme' , distmat_txt )
    out_tree = foldseek2tree.postprocess(out_tree)
    out_tree = madroot(out_tree)
    tre = toytree.tree(out_tree)
    #species_mapper = 
    tipnames = tre.get_tip_labels()
    unidf = AFDB_tools.grab_entries(tipnames)
    lineages = treescore.make_lineages(unidf)
    discretized_levels = 10
    maxnodesize = 15
    taxlabels = dict(zip(unidf['query'] ,unidf.Organism) )
    taxnames = [ i+ ' '+taxlabels[i] if i in taxlabels else i for i in tipnames]
    tre = treescore.label_leaves(tre,lineages)
    treevals = tre.get_node_values('score', True, True)
    print(treevals)
    color_vals = list(red.range_to(blue, discretized_levels+1))
    overlap = treescore.getTaxOverlap(tre.treenode)
    treevals = tre.get_node_values('size', True, True)
    print(treevals)
    maxval = np.amax(treevals)
    minval =0
    bins = [minval + i*(maxval-minval)/discretized_levels for i in range(discretized_levels+1)]
    inds = np.digitize(treevals, bins)
    colors = [ color_vals[i-1].hex_l for i in list(inds)]
    sizes = [ (i / discretized_levels) * maxnodesize for i in list(inds) ]
    print(t)
    standard_treedraw(tre , sizes= None , colors=None , tiplabels = taxnames , save_file = out_tree + '.svg')
    standard_treedraw(tre  , sizes= sizes , colors=colors , tiplabels = taxnames )
    

./examples/swiss_trees_updated/


NameError: name 'pd' is not defined

In [6]:
import traceback
print(swiss_toytrees.keys())

import seaborn
treescores_st = []
treescores_struct = []
rfdistances = []


for t in swiss_toytrees:
    folder = swiss_toytrees[t]['path'].replace( 'pruned.nhx' , 'structs/' )
    
    tmres = folder +'aln_score.tsv'
    res = pd.read_table(tmres, header = None ,delim_whitespace=True)
    res[0] = res[0].map(lambda x :x.replace('.pdb', ''))
    res[1] = res[1].map(lambda x :x.replace('.pdb', ''))
    ids = list( set(list(res[0].unique()) + list(res[1].unique())))
    swisst = swiss_toytrees[t]
    ids = list(set(ids).intersection(set(swisst['labels'])))
    labelorder = [ l for l in swisst['labels'] if l in ids ]
    positions = [ k for i,k in enumerate(list(swisst['coords'][:,1])) if swisst['labels'][i] in ids ]
    pos = { protid : i for i,protid in enumerate(ids) }
    self_dists = res[res[0] == res[1]]
    self_distmap = dict(zip(self_dists[0] , self_dists[2] ) )    
    kernel_distmat = np.zeros((len(pos), len(pos)))
    distmat = np.zeros((len(pos), len(pos)))
    for idx,row in res.iterrows():
        kernel_distmat[pos[row[0]] , pos[row[1]]] = foldseek2tree.kernelfun(self_distmap[row[0]] , self_distmap[row[1]] , row[2])
        distmat[pos[row[0]] , pos[row[1]]]= row[3]
    distmat = 1- distmat
    distmat_txt_kernel = foldseek2tree.distmat_to_txt( ids , distmat , folder )
    out_tree_kernel = foldseek2tree.runFastme( 'fastme ' , distmat_txt_kernel )
    out_tree_kernel = foldseek2tree.postprocess(out_tree_kernel)
    out_tree_kernel = madroot(out_tree_kernel)
    tre = toytree.tree(out_tree_kernel)
    
    tipnames = tre.get_tip_labels()
    unidf = AFDB_tools.grab_entries(ids)
    lineages = treescore.make_lineages(unidf)
    discretized_levels = 10
    maxnodesize = 15
    
    taxlabels = dict(zip(unidf['query'] ,unidf.Organism) )
    taxnames = [ i+ ' '+taxlabels[i] if i in taxlabels else i for i in tipnames]
    tre = treescore.label_leaves(tre,lineages)
    color_vals = list(red.range_to(blue, discretized_levels+1))
    overlap = treescore.getTaxOverlap(tre.treenode)
    treevals = tre.get_node_values('size', True, True)
    
    try:
        rf = tre.treenode.robinson_foulds(swisst['tree'].treenode )
        print( 'RF dist:' , rf )
        rfdistances.append(rf)
    
    except:
        print('rferr')
        rfdistances.append(None)

            
    print('struct score: ', tre.treenode.score, 'ST score: ' , swisst['tree'].treenode.score )

    treescores_struct.append( tre.treenode.score)
    treescores_st.append( swisst['tree'].treenode.score  )
    
    maxval = np.amax(treevals)
    minval =0
    bins = [minval + i*(maxval-minval)/discretized_levels for i in range(discretized_levels+1)]
    inds = np.digitize(treevals, bins)
    colors = [ color_vals[i-1].hex_l for i in list(inds)]
    sizes = [ (i / discretized_levels) * maxnodesize for i in list(inds) ]
    standard_treedraw(tre ,fixed_order=labelorder, fixed_position=positions 
                      ,colors = colors , sizes = sizes, tiplabels=taxnames,  ts = 'o')
    
plt.scatter( x = treescores_struct , y = treescores_st  , c = 'b', ls = '-')
plt.plot( [0,1500] , [0,1500])
plt.xlabel('structure')
plt.ylabel('swisstree')
plt.show()

dict_keys([])


NameError: name 'plt' is not defined